<a href="https://colab.research.google.com/github/ykato27/Optimization/blob/main/Python%26%E6%95%B0%E7%90%86%E6%9C%80%E9%81%A9%E5%8C%96%E3%82%92%E7%94%A8%E3%81%84%E3%81%A6%E6%9C%80%E9%81%A9%E3%81%AA%E6%A0%84%E9%A4%8A%E7%B4%A0%E3%81%AE%E7%8C%AE%E7%AB%8B%E3%82%92%E7%AE%97%E5%87%BA%E3%81%99%E3%82%8B%E3%83%AC%E3%82%B7%E3%83%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pulp

In [2]:
from pulp import *

problem = LpProblem('simple_problem', LpMaximize)
x = LpVariable('x')
y = LpVariable('y')

problem += x + y

problem += -4*x + 12 >= y
problem += -1/2*x+4 >= y

status = problem.solve()
print(LpStatus[status])

print("x:",x.value())
print("y:",y.value())

Optimal
x: 2.2857143
y: 2.8571429


#### 主婦問題を解く

In [3]:
import pandas as pd
dffoods = pd.read_csv("https://api.axross-recipe.com/attachments/3216/url")
dffoods

,food,Calories,Protein,Calcium,Iron,Vitamin A,Thiamine (Vitamin B1),Riboflavin (Vitamin B2),Niacin,Ascorbic Acid (Vitamin C)
0,Wheat Flour (Enriched),44.7,1411,2.0,365,0.0,55.4,33.3,441,0
1,Macaroni,11.6,418,0.7,54,0.0,3.2,1.9,68,0
2,Wheat Cereal (Enriched),11.8,377,14.4,175,0.0,14.4,8.8,114,0
3,Corn Flakes,11.4,252,0.1,56,0.0,13.5,2.3,68,0
4,Corn Meal,36.0,897,1.7,99,30.9,17.4,7.9,106,0
...,...,...,...,...,...,...,...,...,...,...
72,Chocolate,8.0,77,1.3,39,0.0,0.9,3.4,14,0
73,Sugar,34.9,0,0.0,0,0.0,0.0,0.0,0,0
74,Corn Syrup,14.7,0,0.5,74,0.0,0.0,0.0,5,0
75,Molasses,9.0,0,10.3,244,0.0,1.9,7.5,146,0


In [4]:
dfnutrient_lowerlimit = pd.read_csv("https://api.axross-recipe.com/attachments/3217/url")
dfnutrient_lowerlimit

,Nutrient,Intake,Unit
0,Calories,3.0,kilocalories
1,Protein,70.0,grams
2,Calcium,0.8,grams
3,Iron,12.0,milligrams
4,Vitamin A,5.0,1000IU
5,Thiamine (Vitamin B1),1.8,milligrams
6,Riboflavin (Vitamin B2),2.7,milligrams
7,Niacin,18.0,milligrams
8,Ascorbic Acid (Vitamin C),75.0,milligrams


In [5]:
# 最小化問題なのでLpMinimizeを使います
problem = LpProblem('Stiger Diet', LpMinimize)

# food_amount、食料xに何ドル使うか？という変数のリストです。これから線形最適化を使い、これら変数の最適な値を算出することになります。
# food_used[0]はWheat Flour, food_used[1]はMacaroni...という形で対応します。
# また、必要ではないですが、lowBound=0を入れて、これら変数は必ず0以上であるという条件を追加しています。
food_amount = [LpVariable('food_{0}'.format(i), lowBound=0) for i in range(len(dffoods))]

# 目的関数を設定しています。lpSumは1つのリストの総和を求めるための関数です。（numpyのsumを使っても問題ないです。）
# food_amountはその食料に何ドル使うか？の変数なので、単純に和をとれば食料費の総額になります。
problem += lpSum(food_amount)

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [6]:
# 制約を設定しています。lpDotは2つのリストの内積を求める関数になっています。これで、購入総額が目的関数に設定されたことになります。
for k,v in dfnutrient_lowerlimit.iterrows():
    problem += lpDot(food_amount, dffoods[v.Nutrient]) >= v.Intake

In [7]:
#下記のコードは、10*1 + 20*5 + 30*10 = 410を返す
ans = lpDot([10,20,30], [1,5,10])
ans

LpAffineExpression()

In [8]:
status = problem.solve()
print(LpStatus[status])

# 年間コストを算出するため、最適値に365を乗じていっています。
annual_cost = 0
for i, fa in enumerate(food_amount):
    if fa.value() > 0:
        print("{0} : ${1}".format(dffoods.food[i], fa.value()))
        annual_cost += 365 * fa.value()
        
print("上記がコスト最小となる最適な食材の組み合わせで、年間{0}ドルで済む".format(annual_cost))

Optimal
Wheat Flour (Enriched) : $0.029519062
Liver (Beef) : $0.0018925573
Cabbage : $0.011214435
Spinach : $0.0050076605
Navy Beans, Dried : $0.061028564
上記がコスト最小となる最適な食材の組み合わせで、年間39.661731762ドルで済む
